In [119]:
import pandas as pd
import numpy as np

In [120]:
# Data Preprocessing (5 csv is used)
# uk bank holidays
uk_bank_holiday_data = pd.read_csv('data/uk_bank_holidays.csv')
uk_bank_holiday_data.head()

,Bank holidays,Type
0,2012-12-26,Boxing Day
1,2012-12-25,Christmas Day
2,2012-08-27,Summer bank holiday
3,2012-05-06,Queen?s Diamond Jubilee (extra bank holiday)
4,2012-04-06,Spring bank holiday (substitute day)


In [121]:
# Add new column 'holiday'
uk_bank_holiday_data['holiday'] = 1
uk_bank_holiday_data.head()

,Bank holidays,Type,holiday
0,2012-12-26,Boxing Day,1
1,2012-12-25,Christmas Day,1
2,2012-08-27,Summer bank holiday,1
3,2012-05-06,Queen?s Diamond Jubilee (extra bank holiday),1
4,2012-04-06,Spring bank holiday (substitute day),1


In [122]:
# drop Type
uk_bank_holiday_data = uk_bank_holiday_data.drop(['Type'], axis=1)

In [123]:
# Change object class to datetime
uk_bank_holiday_data['Bank holidays'] = pd.to_datetime(uk_bank_holiday_data['Bank holidays'])
uk_bank_holiday_data['Bank holidays'].describe()

C:\Users\ADMIN\miniconda3\envs\power-usage-forecasting-labs\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  This is separate from the ipykernel package so we can avoid doing imports until


count                      25
unique                     25
top       2014-05-05 00:00:00
freq                        1
first     2012-02-01 00:00:00
last      2014-12-26 00:00:00
Name: Bank holidays, dtype: object

In [124]:
date_range = pd.date_range(start='2011-11-01', end='2014-02-28')

In [125]:
# fill in all missing dates
uk_bank_holiday_data= uk_bank_holiday_data.set_index('Bank holidays').reindex(date_range).fillna(int(0)).rename_axis('date').reset_index()

In [126]:
uk_bank_holiday_data.holiday = uk_bank_holiday_data.holiday.astype(int)
uk_bank_holiday_data.holiday.unique()

array([0, 1])

In [127]:
# daily weather - all in degree
weather_daily_data = pd.read_csv('data/weather_daily_darksky.csv')
weather_daily_data.head()

,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,...,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase
0,11.96,2011-11-11 23:00:00,123,fog,9.40,2011-11-11 07:00:00,0.79,3.88,1016.08,2011-11-11 07:00:00,...,10.87,2011-11-11 07:12:14,2011-11-11 19:00:00,2011-11-11 11:00:00,Foggy until afternoon.,2011-11-11 19:00:00,6.48,2011-11-11 23:00:00,2011-11-11 19:00:00,0.52
1,8.59,2011-12-11 14:00:00,198,partly-cloudy-day,4.49,2011-12-11 01:00:00,0.56,3.94,1007.71,2011-12-11 02:00:00,...,8.59,2011-12-11 07:57:02,2011-12-11 14:00:00,2011-12-11 12:00:00,Partly cloudy throughout the day.,2011-12-12 07:00:00,0.11,2011-12-11 20:00:00,2011-12-12 08:00:00,0.53
2,10.33,2011-12-27 02:00:00,225,partly-cloudy-day,5.47,2011-12-27 23:00:00,0.85,3.54,1032.76,2011-12-27 22:00:00,...,10.33,2011-12-27 08:07:06,2011-12-27 14:00:00,2011-12-27 00:00:00,Mostly cloudy throughout the day.,2011-12-27 23:00:00,5.59,2011-12-27 02:00:00,2011-12-28 00:00:00,0.10
3,8.07,2011-12-02 23:00:00,232,wind,3.69,2011-12-02 07:00:00,0.32,3.00,1012.12,2011-12-02 07:00:00,...,7.36,2011-12-02 07:46:09,2011-12-02 12:00:00,2011-12-02 10:00:00,Partly cloudy throughout the day and breezy ov...,2011-12-02 19:00:00,0.46,2011-12-02 12:00:00,2011-12-02 19:00:00,0.25
4,8.22,2011-12-24 23:00:00,252,partly-cloudy-night,2.79,2011-12-24 07:00:00,0.37,4.46,1028.17,2011-12-24 07:00:00,...,7.93,2011-12-24 08:06:15,2011-12-24 15:00:00,2011-12-24 13:00:00,Mostly cloudy throughout the day.,2011-12-24 19:00:00,-0.51,2011-12-24 23:00:00,2011-12-24 20:00:00,0.99


In [128]:
# Take apparent temperature high and low to get mean and also uvIndexTime
kept_columns = ['uvIndexTime', 'apparentTemperatureHigh', 'apparentTemperatureLow']
weather_kept = weather_daily_data[kept_columns]
weather_kept.head()

,uvIndexTime,apparentTemperatureHigh,apparentTemperatureLow
0,2011-11-11 11:00:00,10.87,10.87
1,2011-12-11 12:00:00,5.62,-0.64
2,2011-12-27 00:00:00,10.33,5.52
3,2011-12-02 10:00:00,5.33,3.26
4,2011-12-24 13:00:00,5.02,4.37


In [129]:
# Check for NA
# One missing value here
weather_kept.uvIndexTime.isnull().sum() # 1

1

In [130]:
weather_kept.apparentTemperatureHigh.isnull().sum()

0

In [131]:
weather_kept.apparentTemperatureLow.isnull().sum()

0

In [132]:
weather_kept[weather_kept.isnull().any(axis=1)] #uvIndexTime show NaN

,uvIndexTime,apparentTemperatureHigh,apparentTemperatureLow
846,NaN,10.83,3.36


In [133]:
# Replace NaN with 2014-01-01 shown on apparentTempMinTime
pd.options.mode.chained_assignment = None
weather_kept.loc[846, 'uvIndexTime'] = '2014-01-01'

In [134]:
weather_kept.loc[846, 'uvIndexTime']

'2014-01-01'

In [135]:
weather_kept['date'] = weather_kept.loc[: ,'uvIndexTime'].astype(str).str[:-9]
weather_kept.head()

,uvIndexTime,apparentTemperatureHigh,apparentTemperatureLow,date
0,2011-11-11 11:00:00,10.87,10.87,2011-11-11
1,2011-12-11 12:00:00,5.62,-0.64,2011-12-11
2,2011-12-27 00:00:00,10.33,5.52,2011-12-27
3,2011-12-02 10:00:00,5.33,3.26,2011-12-02
4,2011-12-24 13:00:00,5.02,4.37,2011-12-24


In [136]:
# get mean temperature
temp_mean_col = weather_kept.loc[: , "apparentTemperatureHigh":"apparentTemperatureLow"]
weather_kept['temp_mean'] = temp_mean_col.mean(axis=1)

In [137]:
# drop uvIndexTime, apparentTempHigh, apparentTempLow
weather_kept = weather_kept.drop(['uvIndexTime', 'apparentTemperatureHigh', 'apparentTemperatureLow'], axis=1)
weather_kept.head()

,date,temp_mean
0,2011-11-11,10.870
1,2011-12-11,2.490
2,2011-12-27,7.925
3,2011-12-02,4.295
4,2011-12-24,4.695


In [138]:
# Fill in missing dates
weather_kept['date'] = pd.to_datetime(weather_kept['date'],  errors='coerce')

In [139]:
weather_kept= weather_kept.set_index('date').reindex(date_range).rename_axis('date').reset_index()

In [140]:
weather_kept[weather_kept.isnull().any(axis=1)] # Na

,date,temp_mean
792,2014-01-01,NaN


In [141]:
# replace with mean
weather_kept.loc[792, 'temp_mean'] = (10.83 + 3.36)/2

In [142]:
# daily dataset csv
daily_dataset_csv_data = pd.read_csv('data/daily_dataset.csv')
daily_dataset_csv_data.head()

,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
0,MAC000131,2011-12-15,0.4850,0.432045,0.868,22,0.239146,9.505,0.072
1,MAC000131,2011-12-16,0.1415,0.296167,1.116,48,0.281471,14.216,0.031
2,MAC000131,2011-12-17,0.1015,0.189812,0.685,48,0.188405,9.111,0.064
3,MAC000131,2011-12-18,0.1140,0.218979,0.676,48,0.202919,10.511,0.065
4,MAC000131,2011-12-19,0.1910,0.325979,0.788,48,0.259205,15.647,0.066


In [143]:
daily_dataset_csv_data.day.describe() # 829 only -> instead of 851, na?

count        3510433
unique           829
top       2012-12-18
freq            5541
Name: day, dtype: object

## Getting total usage of whole region
#### - get sum of each household on each day 

In [144]:
total_power = []
# loop through all dates
for date in daily_dataset_csv_data.day.unique():
    # get specific day
    each_day_data = daily_dataset_csv_data.loc[daily_dataset_csv_data['day'] == date]
    # get sum of each day
    sum_energy_per_day = each_day_data['energy_sum'].sum()
    # append to list
    total_power.append([date,sum_energy_per_day])

In [145]:
#population / number of households in London
daily_dataset_csv_data['LCLid'].nunique() 

5566

In [146]:
# To array
arr = np.array(total_power)
# To df
total_power_df = pd.DataFrame(arr, columns=['date', 'total power consumption'])
total_power_df.head()

,date,total power consumption
0,2011-12-15,3101.3840014
1,2011-12-16,3504.5329971
2,2011-12-17,3783.3509991
3,2011-12-18,4125.0359997
4,2011-12-19,4270.0860012


In [147]:
# change date to datetime format
total_power_df['date'] = pd.to_datetime(total_power_df['date'])

In [148]:
# fill in date
total_power_df= total_power_df.set_index('date').reindex(date_range).rename_axis('date').reset_index()

In [149]:
# handle missing value? - use mean
total_power_df['total power consumption'].isnull().sum()

22

In [150]:
total_power_df[total_power_df.isnull().any(axis=1)] # Na are first 22 row of dates

,date,total power consumption
0,2011-11-01,NaN
1,2011-11-02,NaN
2,2011-11-03,NaN
3,2011-11-04,NaN
4,2011-11-05,NaN
5,2011-11-06,NaN
6,2011-11-07,NaN
7,2011-11-08,NaN
8,2011-11-09,NaN
9,2011-11-10,NaN


In [151]:
# Check shape
uk_bank_holiday_data.shape

(851, 2)

In [152]:
total_power_df.shape

(851, 2)

In [153]:
weather_kept.shape

(851, 2)

In [154]:
# Set index
uk_bank_holiday_data = uk_bank_holiday_data.set_index('date')
total_power_df = total_power_df.set_index('date')
weather_kept = weather_kept.set_index('date')

In [155]:
# concat all data
# df = uk_bank_holiday_data + total_power_df + weather_kept
df = pd.concat([uk_bank_holiday_data, total_power_df, weather_kept], axis=1)
df

,holiday,total power consumption,temp_mean
date,,,
2011-11-01,0,NaN,11.450
2011-11-02,0,NaN,14.525
2011-11-03,0,NaN,14.965
2011-11-04,0,NaN,12.855
2011-11-05,0,NaN,10.485
...,...,...,...
2014-02-24,0,51994.5470041,9.505
2014-02-25,0,51423.5080014,5.990
2014-02-26,0,50943.30599460001,7.160


In [156]:
# Remove all data on 22 days
df = df.iloc[22:]
# Check for na again
df.isna().any()

holiday                    False
total power consumption    False
temp_mean                  False
dtype: bool

In [157]:
df.head()

,holiday,total power consumption,temp_mean
date,,,
2011-11-23,0,90.38500010000001,8.460
2011-11-24,0,213.4119999,10.115
2011-11-25,0,303.99299990000003,8.480
2011-11-26,0,420.97599970000005,12.275
2011-11-27,0,444.8830006,7.425


In [158]:
# change power to 2 decimal
df['total power consumption'] = df['total power consumption'].astype(str).astype(float)
df['total power consumption'] = df['total power consumption'].round(2)

In [159]:
# change temperature mean to 2 decimal
df['temp_mean'] = df['temp_mean'].round(2)

In [160]:
# Final data
df

,holiday,total power consumption,temp_mean
date,,,
2011-11-23,0,90.39,8.46
2011-11-24,0,213.41,10.12
2011-11-25,0,303.99,8.48
2011-11-26,0,420.98,12.28
2011-11-27,0,444.88,7.42
...,...,...,...
2014-02-24,0,51994.55,9.51
2014-02-25,0,51423.51,5.99
2014-02-26,0,50943.31,7.16


In [161]:
df.to_csv('preprocessed_data.csv')